In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import PIL
import math
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from random import randint

In [83]:
white = 255
black = 0

def inverse_color(img):
    return PIL.Image.eval(img, lambda val: 255 - val)

def fix_background_color_bug(img):
    colors = sorted(img.getcolors(), key=lambda pair: pair[0], reverse=True)
    replace_color = colors[0][1]
    remove_color = colors[2][1] if colors[2][1] > colors[1][1] else colors[1][1]

    data = np.array(img)
    data[data == remove_color] = replace_color
    return PIL.Image.fromarray(data)

def to_binary(img):
    return PIL.Image.eval(img, lambda val: 255 if val < (256/2) else 0)

In [100]:
binary_image = True


## Rotation, Brightness and Resizing
common_transforms = [
    transforms.RandomRotation(360, fill=black),
    #         transforms.CenterCrop(300),
    transforms.Resize((30, 30)),
]

if not binary_image:
    common_transforms.append(transforms.ColorJitter(brightness=(0.8, 1), contrast=(0.45, 1)))
else:
    common_transforms.append(transforms.Lambda(to_binary))

common_transform = transforms.Compose(common_transforms)


## Adding Grayscale + Inverse color to operators
operators_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Lambda(inverse_color),
    # Randomly scale up and down
#     transforms.RandomAffine(0, scale=(0.9, 1.1), fillcolor=white),
    common_transform,
])

operators_tensor_transform = transforms.Compose([
    operators_transform,
    transforms.ToTensor(),
])

minst_tensor_transform = transforms.Compose([
    common_transform,
    transforms.ToTensor(),
])

In [101]:
# Operators Dataset
operators_dataset = datasets.ImageFolder(root='operators', transform=operators_tensor_transform)

# Show exemple image
datasets.ImageFolder(root='operators', transform=operators_transform)[0][0]

In [102]:
# Minst Dataset
minst_dataset = datasets.MNIST("", transform=minst_tensor_transform, download=True)

# Show example image
datasets.MNIST("", transform=common_transform, download=True)[0][0]

In [103]:
def get_stats(dataset):
    loader = torch.utils.data.DataLoader(dataset, batch_size=50,
                                             shuffle=False, num_workers=2)
    mean = 0.
    std = 0.
    for images, _ in loader:
        batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)

    mean /= len(loader.dataset)
    std /= len(loader.dataset)

    return mean, std

In [104]:
operators_mean, operators_std = get_stats(operators_dataset)
operators_mean, operators_std

(tensor([0.6462]), tensor([0.4699]))

In [105]:
minst_mean, minst_std = get_stats(minst_dataset)
minst_mean, minst_std

(tensor([0.8694]), tensor([0.3303]))

In [106]:
class FusionDataset(datasets.VisionDataset):
    """Custom Dataset for loading CelebA face images"""

    def __init__(self, operators_dataset, minst_dataset):
        self.operators_dataset = operators_dataset
        self.minst_dataset = minst_dataset

    def __getitem__(self, index):
        if index < len(self.operators_dataset):
            tensor, class_index = self.operators_dataset[index]   
            tensor = transforms.Normalize(operators_mean, operators_std)(tensor)
            return tensor, class_index + 10
        else:
            tensor, class_index = self.minst_dataset[index - len(self.operators_dataset)]
            tensor = transforms.Normalize(minst_mean, minst_std)(tensor)
            return tensor, class_index

    def __len__(self):
        return len(self.minst_dataset) + len(self.operators_dataset)
    
def get_loaders(dataset, batch_size=100, validation_split=0.2, shuffle_dataset=True):
    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    split = int(np.floor(validation_split * dataset_size))
    if shuffle_dataset:
        np.random.seed(randint(0, 100))
        np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                               sampler=train_sampler)
    validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                    sampler=valid_sampler)
    
    return train_loader, validation_loader

In [107]:
dataset = FusionDataset(operators_dataset, minst_dataset)

In [108]:
mean, std = get_stats(dataset)
mean, std

(tensor([2.7548e-05]), tensor([1.0000]))

In [109]:
# Creating a Net class object, which consists of 2 convolutional layers, max-pool layers and fully-connected layers
class Conv_Net(nn.Module):
    
    def __init__(self, nb_hidden=50):        
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3)  # the first convolutional layer, which processes the input image
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3)  # the second convolutional layer, which gets the max-pooled set
        self.fc1 = nn.Linear(1152, nb_hidden)  # the first fully-connected layer, which gets flattened max-pooled set
        self.fc2 = nn.Linear(nb_hidden, 15)  # the second fully-connected layer that outputs the result

    # Creating the forward pass
    def forward(self, x):
        
        # The first two layers
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2))
        
        # The second two layers
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2)) 
        
        # Flattening the data set for fully-connected layer
        x = x.view(x.size(0), -1)
    
        # The first fully-connected layer
        x = F.relu(self.fc1(x))
        
        # The second full-connected layer
        x = self.fc2(x)
        
        return x

In [111]:
%%time

model = Conv_Net()
train_loader, test_loader = get_loaders(dataset)

losses = []
test_losses = []

# Defining the optimizer for GD
lr = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr = lr) 

# Defining the criterion to calculate loss
criterion = nn.CrossEntropyLoss()

# Train the model
nb_epochs = 30


for e in range(nb_epochs):
    # Train the input dataset by dividing it into mini_batch_size small datasets
    for train_input, train_target in train_loader:

        # Model computations
        output = model(train_input)
        loss = criterion(output, train_target) 
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        
    print('%dth epoch is finished and the loss is %f' % (e+1, loss))
    losses.append(loss)
    
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    for test_input, test_target in test_loader:
        
        output = model(test_input)
        loss = criterion(output, test_target) 
        
        tmp_acc = 0
        for tensor, target in zip(output, test_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
                
        tmp_acc /= len(output)
        acc += tmp_acc 
                
    acc /= len(test_loader)
    print("test loss: %f, acc: %f" % (loss, acc))
    test_losses.append(loss)

1th epoch is finished and the loss is 0.561357
test loss: 3.942132, acc: 0.589421
2th epoch is finished and the loss is 1.394782
test loss: 0.327267, acc: 0.708347
3th epoch is finished and the loss is 0.324581
test loss: 1.049312, acc: 0.751901
4th epoch is finished and the loss is 0.518378
test loss: 0.732578, acc: 0.789339
5th epoch is finished and the loss is 0.632969
test loss: 3.683100, acc: 0.806198
6th epoch is finished and the loss is 0.540124
test loss: 0.536292, acc: 0.834545
7th epoch is finished and the loss is 0.368760
test loss: 0.083647, acc: 0.841983
8th epoch is finished and the loss is 1.927319
test loss: 0.030339, acc: 0.856281
9th epoch is finished and the loss is 0.010527
test loss: 0.061796, acc: 0.866942
10th epoch is finished and the loss is 0.278621
test loss: 0.035545, acc: 0.862397
11th epoch is finished and the loss is 0.067737
test loss: 0.102889, acc: 0.879008
12th epoch is finished and the loss is 0.373472
test loss: 0.291908, acc: 0.882066
13th epoch is